### Installing and importing the packages

In [0]:
!pip install -r ../requirements.txt

In [0]:
import json
import logging
import os

import torch
import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers.trainer_utils import is_main_process

from contract_nli.conf import load_conf
from contract_nli.dataset.dataset import load_and_cache_examples, load_and_cache_features
from contract_nli.dataset.encoder import SPAN_TOKEN
from contract_nli.evaluation import evaluate_all
from contract_nli.model.identification_classification import MODEL_TYPE_TO_CLASS, update_config
from contract_nli.model.classification import BertForClassification
from contract_nli.postprocess import format_json
from contract_nli.predictor import predict, predict_classification
from contract_nli.trainer import Trainer, setup_optimizer
from contract_nli.utils import set_seed, distributed_barrier

# import mlflow

logger = logging.getLogger(__name__)

### Global parameter settings

In [0]:
conf_path = "./../data/conf_base.yml"
output_dir = '/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5'
local_rank = -1
shared_filesystem = -1


In [0]:
from contract_nli.conf import load_conf

conf: dict = load_conf(conf_path)

print(json.dumps(conf, sort_keys=True, indent=4))

In [0]:
# Setup CUDA, GPU & distributed training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device, n_gpu, conf["num_epochs"])

In [0]:
# if this is a main process in a node
local_main = is_main_process(local_rank)
# if this is a main process in the whole distributed training
all_main = local_rank == -1 or torch.distributed.get_rank() == 0
# if this is a main process on a filesystem
fs_main = (shared_filesystem and all_main) or ((not shared_filesystem) and local_main)

In [0]:
# Setup CUDA, GPU & distributed training
if local_rank == -1 or conf['no_cuda']:
    device = torch.device("cuda" if torch.cuda.is_available() and not conf['no_cuda'] else "cpu")
    n_gpu = 0 if conf['no_cuda'] else torch.cuda.device_count()
else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    torch.distributed.init_process_group(backend="nccl")
    n_gpu = 1

# if this is a main process in a node
local_main = is_main_process(local_rank)
# if this is a main process in the whole distributed training
all_main = local_rank == -1 or torch.distributed.get_rank() == 0
# if this is a main process on a filesystem
fs_main = (shared_filesystem and all_main) or ((not shared_filesystem) and local_main)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if local_main else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    local_rank,
    device,
    n_gpu,
    bool(local_rank != -1),
    conf['fp16'],
)
# Set the verbosity to info of the Transformers logger (on main process only):
if is_main_process(local_rank):
    transformers.utils.logging.set_verbosity_info()
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()
# Set seed
set_seed(conf['seed'])  #42
# fs_main True local_rank -1

### Loading base Tokenizer and BERT model

In [0]:
with distributed_barrier(not fs_main, local_rank != -1):
  config = AutoConfig.from_pretrained(
      conf['config_name'] if conf['config_name'] else conf['model_name_or_path'],
      cache_dir=conf['cache_dir']
  )
  tokenizer = AutoTokenizer.from_pretrained(
      conf['tokenizer_name'] if conf['tokenizer_name'] else conf['model_name_or_path'],
      do_lower_case=conf['do_lower_case'],
      cache_dir=conf['cache_dir'],
      use_fast=False
  )
  if conf['task'] == 'identification_classification':
      config = update_config(
          config, impossible_strategy='ignore',
          class_loss_weight=conf['class_loss_weight'])
      model = MODEL_TYPE_TO_CLASS[config.model_type].from_pretrained(
          conf['model_name_or_path'],
          from_tf=bool(".ckpt" in conf['model_name_or_path']),
          config=config,
          cache_dir=conf['cache_dir']
      )
  else:
      model = BertForClassification.from_pretrained(
          conf['model_name_or_path'],
          from_tf=bool(".ckpt" in conf['model_name_or_path']),
          config=config,
          cache_dir=conf['cache_dir']
      )

logger.info("Training/evaluation parameters %s",
          {k: v for k, v in conf.items() if k != 'raw_yaml'})


### Loading Train Dataset and converting to features

In [0]:
if conf['fp16']:
        try:
            import apex
            apex.amp.register_half_function(torch, "einsum")
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

with distributed_barrier(not fs_main, local_rank != -1):
    examples = load_and_cache_examples(
        conf['train_file'],
        local_rank=local_rank,
        overwrite_cache=conf['overwrite_cache'],
        cache_dir=conf['cache_dir'],
    )
    if conf['task'] == 'identification_classification':
        n_added_token = tokenizer.add_special_tokens(
            {'additional_special_tokens': tokenizer.additional_special_tokens + [SPAN_TOKEN]})
        if n_added_token == 0:
            logger.warning(
                f'SPAN_TOKEN "{SPAN_TOKEN}" was not added. You can safely ignore'
                ' this warning if you are retraining a model from this train.py')
        else:
            span_token_id = tokenizer.additional_special_tokens_ids[
                tokenizer.additional_special_tokens.index(SPAN_TOKEN)]
            logger.warning(
                f'SPAN_TOKEN "{SPAN_TOKEN}" was added as "{span_token_id}". You can safely ignore'
                ' this warning if you are training a model from pretrained LMs.')
    if conf['symbol_based_hypothesis']:
        hypothesis_symbols = sorted(set([e.hypothesis_symbol for e in examples]))
        n_added_token = tokenizer.add_special_tokens(
            {'additional_special_tokens': tokenizer.additional_special_tokens + hypothesis_symbols})
        if n_added_token == 0:
            logger.warning(
                f'SPAN_TOKEN "{SPAN_TOKEN}" was not added. You can safely ignore'
                ' this warning if you are retraining a model from this train.py')
        else:
            assert n_added_token == len(hypothesis_symbols)
            hypothesis_symbol_dic = {
                s: tokenizer.additional_special_tokens_ids[tokenizer.additional_special_tokens.index(s)]
                for s in hypothesis_symbols
            }
            logger.warning(
                f'Hypothesis symbols were added as "{hypothesis_symbol_dic}". '
                'You can safely ignore this warning if you are training a '
                'model from pretrained LMs.')
    model.resize_token_embeddings(len(tokenizer))
    train_dataset = load_and_cache_features(
            conf['train_file'],
            examples,
            tokenizer,
            max_seq_length=conf['max_seq_length'],
            doc_stride=conf.get('doc_stride', None),
            max_query_length=conf['max_query_length'],
            dataset_type=conf['task'],
            symbol_based_hypothesis=conf['symbol_based_hypothesis'],
            threads=None,
            local_rank=local_rank,
            overwrite_cache=conf['overwrite_cache'],
            labels_available=True,
            cache_dir=conf['cache_dir'],
        )[0]

### Loading Dev Dataset and converting to features

In [0]:
if conf['dev_file'] is not None:
    with distributed_barrier(not fs_main, local_rank != -1):
        dev_examples = load_and_cache_examples(
            conf['dev_file'],
            local_rank=local_rank,
            overwrite_cache=conf['overwrite_cache'],
            cache_dir=conf['cache_dir']
        )
        dev_dataset, dev_features = load_and_cache_features(
            conf['dev_file'],
            dev_examples,
            tokenizer,
            max_seq_length=conf['max_seq_length'],
            doc_stride=conf.get('doc_stride', None),
            max_query_length=conf['max_query_length'],
            dataset_type=conf['task'],
            symbol_based_hypothesis=conf['symbol_based_hypothesis'],
            threads=None,
            local_rank=local_rank,
            overwrite_cache=conf['overwrite_cache'],
            labels_available=True,
            cache_dir=conf['cache_dir']
        )

else:
    dev_dataset, dev_examples, dev_features = None, None, None

### Training : Finetuning "bert-base-uncased"

In [0]:
optimizer = setup_optimizer(
        model, learning_rate=conf['learning_rate'], epsilon=conf['adam_epsilon'],
        weight_decay=conf['weight_decay'])
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    optimizer=optimizer,
    task=conf['task'],
    output_dir=output_dir,
    per_gpu_train_batch_size=conf['per_gpu_train_batch_size'],
    num_epochs=conf['num_epochs'],
    max_steps=conf['max_steps'],
    dev_dataset=dev_dataset,
    valid_steps=conf['valid_steps'],
    per_gpu_dev_batch_size=conf['per_gpu_eval_batch_size'],
    gradient_accumulation_steps=conf['gradient_accumulation_steps'],
    warmup_steps=conf['warmup_steps'],
    max_grad_norm=conf['max_grad_norm'],
    n_gpu=n_gpu,
    local_rank=local_rank,
    fp16=conf['fp16'],
    fp16_opt_level=conf['fp16_opt_level'],
    device=device,
    save_steps=conf['save_steps'])
trainer.deploy()
trainer.train()

In [0]:
output_dir, device, trainer

### Logging trainer and saving Model Artifacts

In [0]:
# Setup CUDA, GPU & distributed training
if local_rank != -1:
    trainer.local_rank = -1

if conf['early_stopping'] and os.path.exists(trainer.best_checkpoint_dir):
    logger.info(f"Loading best model from {trainer.best_checkpoint_dir}")
#     trainer.load(trainer.best_checkpoint_dir, map_location='cpu')
    trainer.load(trainer.best_checkpoint_dir)
    trainer.deploy()
    model = trainer.model

logger.info("Saving model checkpoint to %s", output_dir)
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
# Take care of distributed/parallel training
model_to_save = model.module if hasattr(model, "module") else model
# model_to_save.to(device)
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
with open(os.path.join(output_dir, "conf.yml"), 'w') as fout:
    fout.write(conf['raw_yaml'])


In [0]:
output_dir

In [0]:
len(dev_examples)

In [0]:
device

In [0]:
# from contract_nli.predictor import predict, predict_classification
# # model = BertForClassification.from_pretrained(output_dir).to("cuda:0")
# model = trainer.model

# print(conf['task'])

# all_results = predict(
#             model, dev_dataset, dev_examples, dev_features,
#             per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
#             device=device, n_gpu=n_gpu,
#             weight_class_probs_by_span_probs=conf['weight_class_probs_by_span_probs'])

### Getting predictions on Dev Dataset

In [0]:
from contract_nli.postprocess import format_json
logger.info("Evaluate the on validation data")
if conf['task'] == 'identification_classification':
    all_results = predict(
        model, dev_dataset, dev_examples, dev_features,
        per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
        device=device, n_gpu=n_gpu,
        weight_class_probs_by_span_probs=conf['weight_class_probs_by_span_probs'])
else:
    all_results = predict_classification(
        model, dev_dataset, dev_features,
        per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
        device=device, n_gpu=n_gpu)
print(len(all_results), len(dev_examples))

In [0]:
from typing import List, Union, Optional
import collections
from typing import List, Union, Optional

import numpy as np
from scipy.special import softmax

from contract_nli.dataset.encoder import IdentificationClassificationFeatures
from contract_nli.dataset.loader import ContractNLIExample, NLILabel


class IdentificationClassificationPartialResult:
    def __init__(self, unique_id, class_logits, span_logits):
        self.class_logits = class_logits
        self.span_logits = span_logits
        self.unique_id = unique_id


class IdentificationClassificationResult:
    def __init__(self, data_id, class_probs, span_probs):
        self.class_probs = class_probs
        self.span_probs = span_probs
        self.data_id = data_id


class ClassificationResult:
    def __init__(self, data_id, class_probs):
        self.class_probs = class_probs
        self.data_id = data_id
        
        
def format_json(
        all_examples: List[ContractNLIExample],
        all_results: List[Union[IdentificationClassificationResult, ClassificationResult]]
        ) -> List[dict]:

    data_id_to_result = {}
    for result in all_results:
        data_id_to_result[result.data_id] = result

    documents = dict()
    for example_index, example in enumerate(all_examples):
        if example_index not in [84, 85, 86]:
            if example.document_id not in documents:
                documents[example.document_id] = {
                    'id': example.document_id,
                    'file_name': example.file_name,
                    'text': example.context_text,
                    'spans': example.spans,
                    'annotation_sets': [{
                        'user': 'prediction',
                        'mturk': False,
                        'annotations': dict()
                    }]
                }
            assert len(example.spans) == len(example.splits)
            print(example_index)
            print(all_results[0])
            if example.data_id not in data_id_to_result:
                assert isinstance(all_results[0], ClassificationResult)
                continue
            prediction = data_id_to_result[example.data_id]
            d = {
                'choice': NLILabel(
                    np.argmax(prediction.class_probs)).to_anno_name(),
                'class_probs': {
                    NLILabel(i).to_anno_name(): float(p)
                    for i, p in enumerate(prediction.class_probs)
                },
            }
            if isinstance(prediction, IdentificationClassificationResult):
                d.update({
                    'spans': np.where(prediction.span_probs[:, 1] > 0.5)[0].tolist(),
                    'span_probs': prediction.span_probs[:, 1].tolist()
                })
            documents[example.document_id]['annotation_sets'][0]['annotations'][example.hypothesis_id] = d

        if isinstance(all_results[0], IdentificationClassificationResult):
            all_hypothesis_ids = [tuple(sorted(document['annotation_sets'][0]['annotations'].keys())) for document in documents.values()]
            print(len(set(all_hypothesis_ids)))
            assert len(set(all_hypothesis_ids)) == 1
    return sorted(documents.values(), key=lambda d: d['id'])

In [0]:
result_json = format_json(dev_examples, all_results)

### Evaluating the Predictions against True Answers

In [0]:
from collections import defaultdict
from typing import Dict, List, Union

import numpy as np
import sklearn.metrics
from scipy.stats import hmean

from contract_nli.dataset.loader import NLILabel, ContractNLIExample
from contract_nli.postprocess import IdentificationClassificationResult, ClassificationResult


def evaluate_predicted_spans(y_true, y_pred) -> Dict[str, float]:
    if y_true.sum() == 0:
        # do not use zero_division=np.nan because it cannot distinguish
        # zero divisions from y_true and y_pred in f1_score
        recall = np.nan
        f1 = np.nan
    else:
        recall = sklearn.metrics.recall_score(y_true, y_pred)
        f1 = sklearn.metrics.f1_score(y_true, y_pred, zero_division=0)

    return {
        'precision': sklearn.metrics.precision_score(y_true, y_pred, zero_division=0),
        'recall': recall,
        'f1': f1,
        'accuracy': sklearn.metrics.accuracy_score(y_true, y_pred),
    }


def precision_at_recall(y_true, y_prob, recall: float):
    assert 0. <= recall <= 1.0
    if len(y_true) == 0 or np.sum(y_true) == 0:
        return np.nan
    threshs = np.sort(np.unique(y_prob))[::-1]
    # (len(np.unique(y_prob)), len(y_prob)) where first axis show prediction at different thresh
    y_preds = y_prob[None, :] >= threshs[:, None]
    recalls = np.logical_and(y_true[None, :], y_preds).sum(axis=1) / np.sum(y_true)
    # check that recalls are monotonically increasing
    assert np.all(recalls == np.sort(recalls))
    # because of >= relationship, there exist at least one thresh that gives
    # recall score of 1.0
    thresh = threshs[np.where(recalls >= recall)[0][0]]
    y_pred = y_prob >= thresh
    return sklearn.metrics.precision_score(y_true, y_pred, zero_division=0.)


def evaluate_spans(y_true, y_prob) -> Dict[str, float]:
    assert y_prob.ndim == 1
    assert y_true.ndim == 1
    assert len(y_true) == len(y_prob)
    metrics = evaluate_predicted_spans(y_true, y_prob > 0.5)
    metrics.update({
        'roc_auc': sklearn.metrics.roc_auc_score(y_true, y_prob),
        'map': sklearn.metrics.average_precision_score(y_true, y_prob),
        'precision@recall80': precision_at_recall(y_true, y_prob, 0.8),
        'precision@recall90': precision_at_recall(y_true, y_prob, 0.9)
    })
    return metrics


def predict_at_k(y_prob, k):
    y_pred = np.zeros_like(y_prob)
    for j in np.argsort(y_prob)[::-1][:k]:
        y_pred[j] = 1
    assert y_pred.sum() == min(k, len(y_pred))
    return y_pred


def evaluate_class(y_true, y_prob) -> Dict[str, float]:
    assert y_prob.ndim == 2 and y_prob.shape[1] == 3
    assert y_true.ndim == 1
    assert len(y_true) == len(y_prob)
    y_pred = np.argmax(y_prob, axis=1)
    metrics = {
        'accuracy': sklearn.metrics.accuracy_score(y_true, y_pred)
    }
    for label in (NLILabel.YES, NLILabel.NO):
        ln = label.name.lower()
        _y_true = y_true == label.value
        _y_pred = y_pred == label.value
        if _y_true.sum() == 0:
            # do not use zero_division=np.nan because it cannot distinguish
            # zero divisions from y_true and y_pred in f1_score
            recall = np.nan
            f1 = np.nan
        else:
            recall = sklearn.metrics.recall_score(_y_true, _y_pred)
            f1 = sklearn.metrics.f1_score(_y_true, _y_pred, zero_division=0)
        metrics.update({
            f'precision_{ln}': sklearn.metrics.precision_score(_y_true, _y_pred, zero_division=0),
            f'recall_{ln}': recall,
            f'f1_{ln}': f1,
        })
    for m in ('precision', 'recall', 'f1'):
        m_e = metrics[f'{m}_{NLILabel.YES.name.lower()}']
        m_c = metrics[f'{m}_{NLILabel.NO.name.lower()}']
        if np.isnan(m_e) or np.isnan(m_c):
            metrics[f'{m}_mean'] = np.nan
            metrics[f'{m}_hmean'] = np.nan
        else:
            metrics[f'{m}_mean'] = np.mean((m_e, m_c))
            metrics[f'{m}_hmean'] = hmean((m_e, m_c))

    return metrics


def _macro_average(dicts: List[Dict[str, float]]):
    ret = dict()
    for k in dicts[0].keys():
        vals = [d[k] for d in dicts if not np.isnan(d[k])]
        ret[k] = sum(vals) / float(len(vals))
    return ret


def remove_NOT_KNOWN(y_pred):
    assert y_pred.shape[1] == 3
    y_bin = y_pred[:, [NLILabel.NO.value, NLILabel.YES.value]]
    y_bin = np.where(np.tile(np.sum(y_bin, axis=1, keepdims=True), [1, 2]) == 0,
                     0.5,
                     y_bin / np.sum(y_bin, axis=1, keepdims=True))
    y_pred = np.zeros((len(y_pred), 3), dtype=y_pred.dtype)
    y_pred[:, [NLILabel.NO.value, NLILabel.YES.value]] = y_bin
    return y_pred


def evaluate_all(
        dataset: dict,
        results: List[dict],
        ks: List[int],
        task: str
        ) -> dict:
    assert task in ['identification_classification', 'classification', 'identification']
    id_to_result = {r['id']: r for r in results}
    label_ids = sorted(results[0]['annotation_sets'][0]['annotations'].keys())
    class_names = [NLILabel(i).to_anno_name() for i in range(len(NLILabel))]
    assert label_ids == sorted(dataset['labels'].keys()) or task == 'classification'
    if task in ['identification_classification', 'identification']:
        span_probs = defaultdict(list)
        span_labels = defaultdict(list)
    if task in ['identification_classification', 'classification']:
        class_probs = defaultdict(list)
        class_labels = defaultdict(list)
    for document in dataset['documents']:
        result = id_to_result[document['id']]['annotation_sets'][0]['annotations']
        annotations = document['annotation_sets'][0]['annotations']
        for label_id in label_ids:
            if task == 'classification' and label_id not in result:
                continue
            if task in ['identification_classification', 'classification']:
                class_labels[label_id].append(NLILabel.from_str(annotations[label_id]['choice']).value)
                class_probs[label_id].append(
                    np.array([result[label_id]['class_probs'][n] for n in class_names]))
            if task in ['identification_classification', 'identification']:
                # FIXME: this calculates precision optimistically
                if NLILabel.from_str(annotations[label_id]['choice']) != NLILabel.NOT_KNOWN:
                    span_label = np.zeros(len(document['spans']))
                    for s in annotations[label_id]['spans']:
                        span_label[s] = 1
                    span_labels[label_id].append(span_label)
                    print(result[label_id])
                    span_probs[label_id].append(np.array(result[label_id]['span_probs']))
    if task in ['identification_classification', 'classification']:
        binary_label_ids = [
            l for l in label_ids
            if NLILabel.NO.value in class_labels[l] and
               NLILabel.YES.value in class_labels[l]]
        # this is not necessarily true with some training dataset
        # but we have to assume this for our evaluation to be a fair comparison
        if not set(class_probs.keys()).issuperset(set(binary_label_ids)):
            raise ValueError(
                'Some label ids are not in prediction when they are valid label '
                f'ids. Pred: {class_probs.keys()}, Dataset: {binary_label_ids}')
    if task in ['identification_classification', 'identification']:
        preds_at_ks = {
            k: {label_id: [predict_at_k(y_prob, k) for y_prob in y_probs]
                for label_id, y_probs in span_probs.items()}
            for k in ks
        }

    metrics = dict()

    metrics['micro_label_micro_doc'] = dict()
    if task in ['identification_classification', 'classification']:
        metrics['micro_label_micro_doc']['class_binary'] = evaluate_class(
            np.concatenate([np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value]
                            for l in binary_label_ids]),
            remove_NOT_KNOWN(
                np.vstack([np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]
                           for l in binary_label_ids]))
        )
    if task == 'identification_classification':
        metrics['micro_label_micro_doc']['class'] = evaluate_class(
            np.concatenate([class_labels[l] for l in label_ids]),
            np.vstack([np.stack(class_probs[l]) for l in label_ids])
        )
    if task in ['identification_classification', 'identification']:
        y_true = np.concatenate([l for k in label_ids for l in span_labels[k]])
        metrics['micro_label_micro_doc']['span'] = evaluate_spans(
            y_true,
            np.concatenate([l for l in label_ids for l in span_probs[l]])
        )
        for k in ks:
            y_pred = np.concatenate([p for l in label_ids for p in preds_at_ks[k][l]])
            metrics['micro_label_micro_doc']['span'].update({
                f'{n}@{k}': v for n, v in evaluate_predicted_spans(y_true, y_pred).items()
            })
    metrics['macro_label_micro_doc'] = dict()
    if task in ['identification_classification', 'classification']:
        metrics['macro_label_micro_doc']['class_binary'] = _macro_average([
            evaluate_class(
                np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value],
                remove_NOT_KNOWN(np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]))
            for l in binary_label_ids
        ])
    if task == 'identification_classification':
        metrics['macro_label_micro_doc']['class'] = _macro_average([
            evaluate_class(np.array(class_labels[l]), np.stack(class_probs[l]))
            for l in label_ids
        ])
    if task in ['identification_classification', 'identification']:
        metrics['macro_label_micro_doc']['span'] = _macro_average([
            {
                **evaluate_spans(
                    np.concatenate(span_labels[l]),
                    np.concatenate(span_probs[l])),
                **{
                    f'{n}@{k}': v
                    for k in ks
                    for n, v in evaluate_predicted_spans(
                        np.concatenate(span_labels[l]),
                        np.concatenate(preds_at_ks[k][l])).items()
                }
            }
            for l in label_ids
        ])

    if task in ['identification_classification', 'identification']:
        metrics['macro_label_macro_doc'] = dict()
        metrics['macro_label_macro_doc']['span'] = _macro_average([
            _macro_average([
                {
                   **evaluate_spans(span_labels[l][i], span_probs[l][i]),
                   **{
                       f'{n}@{k}': v
                       for k in ks
                       for n, v in evaluate_predicted_spans(
                           span_labels[l][i],
                           preds_at_ks[k][l][i]).items()
                   }
                }
                for i in range(len(span_labels[l]))
            ])
            for l in label_ids
        ])

    metrics['micro_label_macro_doc'] = dict()
    if task == 'identification_classification':
        metrics['micro_label_macro_doc']['class'] = _macro_average([
            evaluate_class(
                np.array([class_labels[l][i] for l in label_ids]),
                np.stack([class_probs[l][i] for l in label_ids]))
            for i in range(len(class_labels[label_ids[0]]))
        ])
    if task in ['identification_classification', 'identification']:
        metrics['micro_label_macro_doc']['span'] = _macro_average([
            evaluate_spans(_l, _p)
            for l in label_ids
            for _l, _p in zip(span_labels[l], span_probs[l])
        ])
        metrics['micro_label_macro_doc']['span'].update({
            key: value
            for k in ks
            for key, value in _macro_average([
                {
                    f'{n}@{k}': v
                    for n, v in evaluate_predicted_spans(_l, _p).items()
                }
                for l in label_ids
                for _l, _p in zip(span_labels[l], preds_at_ks[k][l])
            ]).items()
        })

    metrics['label_wise'] = dict()
    for l in label_ids:
        metrics['label_wise'][l] = dict()
        metrics['label_wise'][l]['micro_doc'] = dict()
        if task in ['identification_classification', 'classification']:
            metrics['label_wise'][l]['micro_doc']['class_binary'] = evaluate_class(
                np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value],
                remove_NOT_KNOWN(np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]))
            if not (NLILabel.NO.value in class_labels[l] and NLILabel.YES.value in class_labels[l]):
                metrics['label_wise'][l]['micro_doc']['class_binary'] = {
                    k: np.nan for k in metrics['label_wise'][l]['micro_doc']['class_binary'].keys()
                }
        if task == 'identification_classification':
            metrics['label_wise'][l]['micro_doc']['class'] = evaluate_class(
                np.array(class_labels[l]), np.stack(class_probs[l]))
        if task in ['identification_classification', 'identification']:
            y_true = np.concatenate(span_labels[l])
            metrics['label_wise'][l]['micro_doc']['span'] = {
                **evaluate_spans(y_true, np.concatenate(span_probs[l])),
                **{
                    f'{n}@{k}': v
                    for k in ks
                    for n, v in evaluate_predicted_spans(
                        y_true, np.concatenate(preds_at_ks[k][l])).items()
                }
            }
            metrics['label_wise'][l]['macro_doc'] = dict()
            metrics['label_wise'][l]['macro_doc']['span'] = _macro_average([
                {
                    **evaluate_spans(span_labels[l][i], span_probs[l][i]),
                    **{
                        f'{n}@{k}': v
                        for k in ks
                        for n, v in evaluate_predicted_spans(
                            span_labels[l][i], preds_at_ks[k][l][i]).items()
                    }
                }
                for i in range(len(span_labels[l]))
            ])
    return metrics


### Saving Model Results and Performance Metrics

In [0]:

with open(os.path.join(output_dir, f'result.json'), 'w') as fout:
        json.dump(result_json, fout, indent=2)
with open(conf['dev_file']) as fin:
    dev_json = json.load(fin)
    
display(result_json)

In [0]:
output_dir, conf['dev_file']

In [0]:
# with open(os.path.join(output_dir, f'result.json'), 'r') as fout:
#     result_json = json.load(fout)
# with open(conf['dev_file']) as fin:
#     dev_json = json.load(fin)


# result_df = spark.read.option("multiline", "true").format("json").load('/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/result.json')
# result_df.show()

with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/result.json', 'r') as f:
    data = f.read()
    result_json = json.loads(data)



result_json

In [0]:
display(result_df)

In [0]:
conf['dev_file']

In [0]:
# dev_df = spark.read.option("multiline", "true").format("json").load('/FileStore/span_nli_bert/data/spannli_dev_data.json')
# dev_df.show()


with open('/dbfs/FileStore/span_nli_bert/data/spannli_dev_data.json', 'r') as p:
    dev_data = p.read()
    dev_json = json.loads(dev_data)
dev_json

In [0]:
display(dev_df)

In [0]:
conf['task']

### Saving for Span Identification classification

In [0]:
metrics = evaluate_all(dev_json, result_json,
                       [1, 3, 5, 8, 10, 15, 20, 30, 40, 50],
                       conf['task'])
display(metrics)

logger.info(f"Results@: {json.dumps(metrics, indent=2)}")
with open(os.path.join(output_dir, f'idtfy_metrics.json'), 'w') as fout:
    json.dump(metrics, fout, indent=2)

In [0]:
metrics = evaluate_all(dev_json, result_json,
                       [1, 3, 5, 8, 10, 15, 20, 30, 40, 50],
                       'classification')
display(metrics)

logger.info(f"Results@: {json.dumps(metrics, indent=2)}")
with open(os.path.join(output_dir, f'cls_metrics.json'), 'w') as fout:
    json.dump(metrics, fout, indent=2)

In [0]:
metrics['micro_label_micro_doc']

In [0]:
metrics['macro_label_micro_doc']

In [0]:
metrics['label_wise']

### Interpreting Experiment Results and Observations

In [0]:
# /dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5
with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/best-checkpoint/config.json', 'r') as f:
    train_config_data = f.read()
    train_config_json = json.loads(train_config_data)

train_config_json

In [0]:
print(train_config)

In [0]:
with open("/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/best-checkpoint/trainer_info.json", 'r') as f:
#     train_info_data = f.read()
    train_info_json = json.load(f)

train_info_json

In [0]:
with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/conf.yml', 'r') as f:
    train_conf_yml = f.read()

    print(train_conf_yml)

In [0]:
with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/config.json', 'r') as f:
    train_config_data = f.read()

train_config_json = json.loads(train_config_data)
train_config_json

In [0]:
import json 
with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED/metrics.json', 'r') as file:
    train_metrics_data = file.read()

train_metrics_json = json.loads(train_metrics_data)
print(train_metrics_data)

In [0]:
train_metrics_json["micro_label_micro_doc"]

In [0]:
train_metrics_json["macro_label_micro_doc"]

In [0]:
import json 
with open('/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/result.json', 'r') as file:
    train_results_data = file.read()

train_results_json = json.loads(train_results_data)
print(train_results_json)